# Orchestrating the Cloud with Kubernetes

In [ ]:
gcloud auth list
gcloud config list project

gcloud config set compute/zone us-central1-b
gcloud container clusters create io
gcloud container clusters get-credentials io

### Task 1. Get the sample code

In [ ]:
gsutil cp -r gs://spls/gsp021/* .
cd orchestrate-with-kubernetes/kubernetes

### Task 2. Quick Kubernetes Demo

In [ ]:
kubectl create deployment nginx --image=nginx:1.10.0
kubectl get pods
kubectl expose deployment nginx --port 80 --type LoadBalancer
kubectl get services
curl http://<External IP>:80

### Task 3. Pods